In [8]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv('data/omea.csv',  index_col=0)

In [4]:
df.head()

,item_id,name,category,price,old_price,sellable_online,link,other_colors,short_description,designer,depth,height,width
0,90420332,FREKVENS,Bar furniture,265.0,No old price,True,https://www.ikea.com/sa/en/p/frekvens-bar-tabl...,No,"Bar table, in/outdoor, 51x51 cm",Nicholai Wiig Hansen,NaN,99.0,51.0
1,368814,NORDVIKEN,Bar furniture,995.0,No old price,False,https://www.ikea.com/sa/en/p/nordviken-bar-tab...,No,"Bar table, 140x80 cm",Francis Cayouette,NaN,105.0,80.0
2,9333523,NORDVIKEN / NORDVIKEN,Bar furniture,2095.0,No old price,False,https://www.ikea.com/sa/en/p/nordviken-nordvik...,No,Bar table and 4 bar stools,Francis Cayouette,NaN,NaN,NaN
3,80155205,STIG,Bar furniture,69.0,No old price,True,https://www.ikea.com/sa/en/p/stig-bar-stool-wi...,Yes,"Bar stool with backrest, 74 cm",Henrik Preutz,50.0,100.0,60.0
4,30180504,NORBERG,Bar furniture,225.0,No old price,True,https://www.ikea.com/sa/en/p/norberg-wall-moun...,No,"Wall-mounted drop-leaf table, ...",Marcus Arvonen,60.0,43.0,74.0


In [6]:
df.old_price.unique()[:20]

array(['No old price', 'SR 2,375', 'SR 295', 'SR 175', 'SR 995',
       'SR 1,385', 'SR 445', 'SR 925', 'SR 2,205', 'SR 895', 'SR 2,435',
       'SR 125', 'SR 1,595', 'SR 2,880', 'SR 940', 'SR 800', 'SR 1,995',
       'SR 3,245', 'SR 8,735', 'SR 2,310'], dtype=object)

In [9]:
def fix_old_price(df):
    '''modify old_price feature'''
    
    if df['old_price']  == 'No old price':
        return df['price']

    elif df['old_price'][-4:] != 'pack':
        return float(str(df['old_price'])[3:].replace(',',''))
        
    else:
        return np.nan

df['discounted'] = (df['old_price'] != 'No old price').astype(int)   
df['old_price'] = df.apply(fix_old_price, axis=1)
df[['price', 'old_price', 'discounted']].head()


,price,old_price,discounted
0,265.0,265.0,0
1,995.0,995.0,0
2,2095.0,2095.0,0
3,69.0,69.0,0
4,225.0,225.0,0


In [10]:
df.groupby('category')[['width', 'height', 'depth']].apply(lambda x: x.notnull().sum())

,width,height,depth
category,,,
Bar furniture,37,35,27
Beds,185,157,83
Bookcases & shelving units,492,442,453
Cabinets & cupboards,256,230,237
Café furniture,21,22,14
Chairs,389,290,278
Chests of drawers & drawer units,125,123,121
Children's furniture,118,98,94
Nursery furniture,93,84,66


In [11]:
df['width_d'] = (df['width'].notnull()).astype(int)
df['height_d'] = (df['height'].notnull()).astype(int)
df['depth_d'] = (df['depth'].notnull()).astype(int)
df[['width', 'height', 'depth', 'width_d', 'height_d', 'depth_d']].head(5)

,width,height,depth,width_d,height_d,depth_d
0,51.0,99.0,NaN,1,1,0
1,80.0,105.0,NaN,1,1,0
2,NaN,NaN,NaN,0,0,0
3,60.0,100.0,50.0,1,1,1
4,74.0,43.0,60.0,1,1,1


In [16]:
df[['width', 'height', 'depth']] = df.groupby(['category'])[['width', 'height', 'depth']].transform(lambda x: x.fillna(x.mean()))

In [17]:
df.groupby('category')[['width', 'height', 'depth']].apply(lambda x: x.notnull().sum())

,width,height,depth
category,,,
Bar furniture,47,47,47
Beds,208,208,208
Bookcases & shelving units,548,548,548
Cabinets & cupboards,292,292,292
Café furniture,26,26,26
Chairs,481,481,481
Chests of drawers & drawer units,125,125,125
Children's furniture,124,124,124
Nursery furniture,97,97,97


In [20]:
cols = ['item_id', 'link',
        'designer']
df2 = df.drop(cols, axis=1)
df2.columns

Index(['name', 'category', 'price', 'old_price', 'sellable_online',
       'other_colors', 'short_description', 'depth', 'height', 'width',
       'discounted', 'width_d', 'height_d', 'depth_d'],
      dtype='object')

In [21]:
df2.isna().sum()

name                  0
category              0
price                 0
old_price            10
sellable_online       0
other_colors          0
short_description     0
depth                 0
height                0
width                 0
discounted            0
width_d               0
height_d              0
depth_d               0
dtype: int64

In [22]:
df2.dropna(inplace=True)
df2.isna().sum()

name                 0
category             0
price                0
old_price            0
sellable_online      0
other_colors         0
short_description    0
depth                0
height               0
width                0
discounted           0
width_d              0
height_d             0
depth_d              0
dtype: int64

In [23]:
df2.head()

,name,category,price,old_price,sellable_online,other_colors,short_description,depth,height,width,discounted,width_d,height_d,depth_d
0,FREKVENS,Bar furniture,265.0,265.0,True,No,"Bar table, in/outdoor, 51x51 cm",46.925926,99.000000,51.000000,0,1,1,0
1,NORDVIKEN,Bar furniture,995.0,995.0,False,No,"Bar table, 140x80 cm",46.925926,105.000000,80.000000,0,1,1,0
2,NORDVIKEN / NORDVIKEN,Bar furniture,2095.0,2095.0,False,No,Bar table and 4 bar stools,46.925926,97.571429,52.945946,0,0,0,0
3,STIG,Bar furniture,69.0,69.0,True,Yes,"Bar stool with backrest, 74 cm",50.000000,100.000000,60.000000,0,1,1,1
4,NORBERG,Bar furniture,225.0,225.0,True,No,"Wall-mounted drop-leaf table, ...",60.000000,43.000000,74.000000,0,1,1,1


In [24]:
df2['discount_amount'] = df2['old_price'] - df2['price']

In [25]:
df2.head()

,name,category,price,old_price,sellable_online,other_colors,short_description,depth,height,width,discounted,width_d,height_d,depth_d,discount_amount
0,FREKVENS,Bar furniture,265.0,265.0,True,No,"Bar table, in/outdoor, 51x51 cm",46.925926,99.000000,51.000000,0,1,1,0,0.0
1,NORDVIKEN,Bar furniture,995.0,995.0,False,No,"Bar table, 140x80 cm",46.925926,105.000000,80.000000,0,1,1,0,0.0
2,NORDVIKEN / NORDVIKEN,Bar furniture,2095.0,2095.0,False,No,Bar table and 4 bar stools,46.925926,97.571429,52.945946,0,0,0,0,0.0
3,STIG,Bar furniture,69.0,69.0,True,Yes,"Bar stool with backrest, 74 cm",50.000000,100.000000,60.000000,0,1,1,1,0.0
4,NORBERG,Bar furniture,225.0,225.0,True,No,"Wall-mounted drop-leaf table, ...",60.000000,43.000000,74.000000,0,1,1,1,0.0


In [27]:
df2['size'] = (np.where(df2['depth_d'] == 1, df2['depth'],1)) *\
(np.where(df2['width_d'] == 1, df2['width'],1)) *\
(np.where(df2['height_d'] == 1, df2['height'],1))


df2[['size', 'width', 'height', 'depth', 'width_d', 'height_d', 'depth_d']].head(10)

,size,width,height,depth,width_d,height_d,depth_d
0,5049.0,51.000000,99.000000,46.925926,1,1,0
1,8400.0,80.000000,105.000000,46.925926,1,1,0
2,1.0,52.945946,97.571429,46.925926,0,0,0
3,300000.0,60.000000,100.000000,50.000000,1,1,1
4,190920.0,74.000000,43.000000,60.000000,1,1,1
5,163800.0,40.000000,91.000000,45.000000,1,1,1
6,209000.0,50.000000,95.000000,44.000000,1,1,1
7,2500.0,50.000000,97.571429,50.000000,1,0,1
8,209000.0,50.000000,95.000000,44.000000,1,1,1
9,1.0,52.945946,97.571429,46.925926,0,0,0


In [28]:
df2.to_csv('clean_OMEA_dataset.csv', index=False)